# How to on Ray: Forecasting
> Run TimeGPT distributedly on top of Ray.

`TimeGPT` works on top of Spark, Dask, and Ray through Fugue. `TimeGPT` will read the input DataFrame and use the corresponding engine. For example, if the input is a Ray DataFrame, `TimeGPT` will use the existing Ray session to run the forecast.


In [ ]:
#| hide
from nixtlats.utils import colab_badge

In [ ]:
#| echo: false
colab_badge('docs/how-to-guides/4_distributed_fcst_ray')

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nixtla/nixtla/blob/main/nbs/docs/how-to-guides/4_distributed_fcst_ray.ipynb)

# Installation 

[Ray](https://www.ray.io/) is an open source unified compute framework to scale Python workloads. As long as Ray is installed and configured, `TimeGPT` will be able to use it. If executing on a distributed Ray cluster, make sure the `nixtlats` library is installed across all the workers.

In addition to Ray, you'll also need to have [Fugue](https://fugue-tutorials.readthedocs.io/) installed. Fugue provides an easy-to-use interface for distributed computing that lets users execute Python code on top of Spark, Dask and Ray. You can install Fugue for Ray using pip. 

In [ ]:
%%capture
pip install "fugue[ray]"

## Executing on Ray

First, instantiate a `NixtlaClient` class. To do this, you will need an API key provided by Nixtla. If you don't have one already, please request yours [here](https://docs.nixtla.io/).

There are different ways to set your API key. Here, we will set it up as an environment variable. Please refer to this [tutorial](https://docs.nixtla.io/docs/setting_up_your_authentication_api_key) to learn more.

In [ ]:
#| hide
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from nixtlats import NixtlaClient

nixtla_client = NixtlaClient() # defaults to os.environ.get("Nixtla_API_KEY")

Start Ray cluster

In [ ]:
import ray
from ray.cluster_utils import Cluster

ray_cluster = Cluster(
    initialize_head=True,
    head_node_args={"num_cpus": 2}
)
ray.init(address=ray_cluster.address, ignore_reinit_error=True)

2024-04-23 16:54:49,846	INFO utils.py:108 -- Overwriting previous Ray address (127.0.0.1:56317). Running ray.init() on this node will now connect to the new instance at 127.0.0.1:58344. To override this behavior, pass address=127.0.0.1:56317 to ray.init().
2024-04-23 16:54:49,848	INFO worker.py:1431 -- Connecting to existing Ray cluster at address: 127.0.0.1:58344...
2024-04-23 16:54:49,849	INFO worker.py:1453 -- Calling ray.init() again after it has already been called.


Python version:,3.10.13
Ray version:,2.6.2


### Forecast

Next, load a pandas DataFrame and then convert it to a Ray dataset. 

In [ ]:
import pandas as pd 

df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short.csv')
df.head()

,unique_id,ds,y
0,BE,2016-12-01 00:00:00,72.00
1,BE,2016-12-01 01:00:00,65.80
2,BE,2016-12-01 02:00:00,59.99
3,BE,2016-12-01 03:00:00,50.69
4,BE,2016-12-01 04:00:00,52.58


In [ ]:
ray_df = ray.data.from_pandas(df)

Now call `NixtlaClient` forecast method. 

In [ ]:
fcst_df = nixtla_client.forecast(ray_df, h=12, freq='H')

2024-04-23 16:57:30,485	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[Repartition]
2024-04-23 16:57:30,486	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-04-23 16:57:30,487	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


- Repartition 1:   0%|          | 0/4 [00:00<?, ?it/s]

Split Repartition 2:   0%|          | 0/4 [00:00<?, ?it/s]

Running 0:   0%|          | 0/4 [00:00<?, ?it/s]

2024-04-23 16:57:31,932	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(add_coarse_key)] -> LimitOperator[limit=1]
2024-04-23 16:57:31,933	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-04-23 16:57:31,934	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


...

Shuffle Map 3:   0%|          | 0/4 [00:00<?, ?it/s]

Shuffle Reduce 4:   0%|          | 0/4 [00:00<?, ?it/s]

Running 0:   0%|          | 0/4 [00:00<?, ?it/s]

Sort Sample 0:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/marianamenchero/miniforge3/envs/nixtlats/lib/python3.10/site-packages/ray/data/_internal/arrow_block.py:128: FutureWarning: promote has been superseded by promote_options='default'.
  return transform_pyarrow.concat(tables)


In [ ]:
fcst_df.to_pandas().head()

,unique_id,ds,TimeGPT
0,FR,2016-12-31 00:00:00,62.130219
1,FR,2016-12-31 01:00:00,56.890831
2,FR,2016-12-31 02:00:00,52.231552
3,FR,2016-12-31 03:00:00,48.888664
4,FR,2016-12-31 04:00:00,46.498367


### Forecast with exogenous variables

Exogenous variables or external factors are crucial in time series forecasting as they provide additional information that might influence the prediction. These variables could include holiday markers, marketing spending, weather data, or any other external data that correlate with the time series data you are forecasting.

For example, if you're forecasting ice cream sales, temperature data could serve as a useful exogenous variable. On hotter days, ice cream sales may increase.

To incorporate exogenous variables in TimeGPT, you'll need to pair each point in your time series data with the corresponding external data.

Let's see an example. First we'll load the data as a pandas DataFrame and then we'll convert it to a Ray dataset. 

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')
df.head()

,unique_id,ds,y,Exogenous1,Exogenous2,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,BE,2016-12-01 00:00:00,72.00,61507.0,71066.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,BE,2016-12-01 01:00:00,65.80,59528.0,67311.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,BE,2016-12-01 02:00:00,59.99,58812.0,67470.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,BE,2016-12-01 03:00:00,50.69,57676.0,64529.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,BE,2016-12-01 04:00:00,52.58,56804.0,62773.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
ray_df = ray.data.from_pandas(df)

To produce forecasts we have to add the future values of the exogenous variables. Let's read this dataset. In this case we want to predict 24 steps ahead, therefore each unique id will have 24 observations.

In [ ]:
future_ex_vars = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-future-ex-vars.csv')
future_ex_vars.head()

,unique_id,ds,Exogenous1,Exogenous2,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,BE,2016-12-31 00:00:00,64108.0,70318.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,BE,2016-12-31 01:00:00,62492.0,67898.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,BE,2016-12-31 02:00:00,61571.0,68379.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,BE,2016-12-31 03:00:00,60381.0,64972.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,BE,2016-12-31 04:00:00,60298.0,62900.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
future_ex_vars_ray = ray.data.from_pandas(future_ex_vars)

Let's call the `forecast` method, adding this information:

In [ ]:
fcst_ex_vars_df = nixtla_client.forecast(df=ray_df, X_df=future_ex_vars_ray, h=24, freq='H', level=[80, 90])

2024-04-23 17:00:37,590	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[Repartition]
2024-04-23 17:00:37,591	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-04-23 17:00:37,591	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


- Repartition 1:   0%|          | 0/4 [00:00<?, ?it/s]

Split Repartition 2:   0%|          | 0/4 [00:00<?, ?it/s]

Running 0:   0%|          | 0/4 [00:00<?, ?it/s]

2024-04-23 17:00:37,688	INFO streaming_executor.py:92 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(add_coarse_key)] -> LimitOperator[limit=1]
2024-04-23 17:00:37,688	INFO streaming_executor.py:93 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-04-23 17:00:37,689	INFO streaming_executor.py:95 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


...

Sort Sample 2:   0%|          | 0/4 [00:00<?, ?it/s]

Shuffle Map 3:   0%|          | 0/4 [00:00<?, ?it/s]

Shuffle Reduce 4:   0%|          | 0/4 [00:00<?, ?it/s]

Running 0:   0%|          | 0/4 [00:00<?, ?it/s]

Sort Sample 0:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
fcst_ex_vars_df.to_pandas().head()

,unique_id,ds,TimeGPT,TimeGPT-lo-90,TimeGPT-lo-80,TimeGPT-hi-80,TimeGPT-hi-90
0,BE,2016-12-31 00:00:00,46.645422,39.503722,41.512494,51.778351,53.787123
1,BE,2016-12-31 01:00:00,56.132756,48.211729,52.26287,60.002641,64.053782
2,BE,2016-12-31 02:00:00,46.858264,40.903685,43.292274,50.424255,52.812844
3,BE,2016-12-31 03:00:00,37.692695,31.257537,34.28142,41.10397,44.127853
4,BE,2016-12-31 04:00:00,38.946163,33.651305,34.671752,43.220573,44.24102


Don't forget to stop Ray once you're done. 

In [ ]:
ray.shutdown()